In [1]:
import pandas, numpy, os, re, datetime, pytz
from ics import Calendar, Event

In [2]:
limite500 = list(range(0, 500))

dataXL = pandas.read_excel("Planning.xlsx", usecols=limite500, nrows=200, skip_blank_lines=True)

dataXL = dataXL.dropna(how='all',axis=0) # Supp ligne NaN
dataXL = dataXL.dropna(how='all',axis=0) # Supp column NaN

In [3]:
my_name =input("Entrez NOM Prenom : ") # -> RAMMAULT Gauthier

col_name = 1 # colonne des prénoms
data = pandas.DataFrame() # cree tableau pandas

while data.empty == True:
    # tant que le tableau est vide...
    # filtre la ligne correspondant au Prenom
    data = dataXL[dataXL.iloc[ : , col_name ].str.contains(my_name, na=False)] #case=False
    col_name = col_name +1

Entrez NOM Prenom : RAMMAULT Gauthier


In [4]:
date_en_titre = False

for element in list(dataXL):
    if type(element) is datetime.datetime:
        date_en_titre = True

if date_en_titre == True:
    print('Date en titre de colonne')
    # la première colonne est une suite de date...
    # conversion tableau Pandas en tableau Numpy
    result = numpy.vstack((numpy.array(list(data)), data.to_numpy()))
    
else :
    print('Datetime à reconstituer')
    jours = dataXL.iloc[0]
    mois = dataXL.iloc[1].replace({'JANVIER': 1, 'FEVRIER': 2, 'MARS': 3, 'AVRIL': 4,
                                   'MAI': 5, 'JUIN':6, 'JUILLET': 7, 'AOUT': 8,
                                   'SEPTEMBRE': 9, 'OCTOBRE': 10, 'NOVEMBRE': 11, 'DECEMBRE': 12,})
    
    now = datetime.datetime.now()
    dates = numpy.array([]) #vide
    
    # boucle de la taille du tableau
    for i in range(dataXL.shape[1]):
        # si c'est un chiffre
        if re.match(r"^[0-9]", str(jours[i])):
            newdate = datetime.datetime(2020, int(mois[i]), int(jours[i]))
            
            # fait la correction decembre 2020 pour janvier 2021
            # avec la protection des mois precedents (sep,oct,nov)
            if int(now.month)-3 > int(newdate.month):
                newdate = newdate + datetime.timedelta(years = 1)
            
            dates = numpy.append(dates, [newdate]) # ajoute la date reconstitué
        else:
            dates = numpy.append(dates, [jours[i]]) # sinon le contenu de la case
    
    # Tableau Numpy
    result = numpy.vstack((dates, data.to_numpy()))

Datetime à reconstituer


In [5]:
try:
    my_folder = 'Planning_'+my_name
    
    os.mkdir(my_folder) # Directory Created
    prevdir = os.getcwd()
    newdir = prevdir+'/'+my_folder
    os.chdir(os.path.expanduser(newdir)) # Directory Change
    
except:
    print("Directory already exists")

In [6]:
def utc(date):
    local_time = pytz.timezone("Europe/Paris")

    naive_datetime = datetime.datetime.strptime (str(date), "%Y-%m-%d %H:%M:%S")

    local_datetime = local_time.localize(naive_datetime, is_dst=None)

    utc_datetime = local_datetime.astimezone(pytz.utc)
    
    return utc_datetime

In [7]:
def creation_event(start, stop, titre, journee):
    #
    # SI date 'start' < Date du jour : ne rien faire. 
    #
    date_simple = str(start)[:10]
    uid_label = 'Work-'+ my_name + date_simple
    
    c = Calendar()
    e = Event(name=titre, begin=utc(start), end=utc(stop), uid=uid_label)

    if journee == True:
        e.make_all_day()
    elif titre != 'Repos':
        e.location = '62 Rue Pierre Charron, 75008 Paris'
    
    c.events.add(e)
    with open('Planning '+date_simple+'.ics', 'w') as my_file:
        my_file.writelines(c)

In [8]:
for i in range(col_name,result.shape[1]):
    # commence après le Prénom jusque fin.
    if str(result[1,i]) == 'nan':
        pass
    
    elif str(result[1,i]) == 'OFF':
        debut = result[0,i] + datetime.timedelta(hours=10)
        fin = debut + datetime.timedelta(hours=1)
        titre = result[1,i]
        #print(f'Du {debut} au {fin} : {titre}')
        creation_event(debut, fin, "Repos", True)
        
    elif re.match(r"^[a-zA-Z]", result[1,i]) :
        debut = result[0,i]
        fin = debut + datetime.timedelta(days=1)
        titre = result[1,i]
        #print(f'Du {debut} au {fin} : {titre}')
        creation_event(debut, fin, titre, False)
        
    elif re.match(r"^[0-9]", result[1,i]) :
        
        # Recuperation des Horaires texte...
        heure1 = int(result[1,i][:2])
        minut1 = int(result[1,i][3:5]) # xxHXX
        
        heure2 = int(result[1,i][6:8]) # xxHxx-XX
        minut2 = int(result[1,i][-2:]) # xxH-xxHXX
        
        # Creation Horaires au format Datetime...
        debut = result[0,i] + datetime.timedelta(hours=heure1) + datetime.timedelta(minutes=minut1)
        fin = result[0,i] + datetime.timedelta(hours=heure2) + datetime.timedelta(minutes=minut2)
        
        # Correction Horaires Minuit+...
        if heure2 < 12:
            fin = fin + datetime.timedelta(days=1)
            
        # Titre Travail + Contenue de la case.
        titre = 'Travail '+result[1,i]
        
        #print(f'Du {debut} au {fin} : {titre}')
        creation_event(debut, fin, titre, False)
    else:
        pass